In [13]:
#Adam Bloodgood

import requests
import json
import pandas as pd
import string
import math
import numpy as np

In [14]:
# Gather details from all of the shots from a specific game ID.

def getShotInfo(play_by_play):

    # Get play-by-play data from the selected game and filter out just the individual plays.

    # Filter out plays that are either shots or goals
    event_types = ["goal", "shot-on-goal"]

    data = pd.DataFrame.from_dict(play_by_play['plays'])
    shots = data[data['typeDescKey'].isin(event_types)]

    shots = shots.reset_index()

    for index, shot in shots.iterrows():
        descriptor = shot['periodDescriptor']
        if descriptor['number'] == 5:
            shots.drop(index, inplace=True)

    # Clean up dataframe to make sense
    shot_details = pd.json_normalize(shots['details'])
    shot_results = shots['typeDescKey'].reset_index()
    homeTeamSide = shots['homeTeamDefendingSide'].reset_index()
    situationCode = shots['situationCode'].reset_index()
    period = shots['periodDescriptor'].reset_index()
    shot_details = pd.concat([shot_results, shot_details, homeTeamSide, period, situationCode], axis=1)
    del shot_details['index']

    return shot_details

def getPlayerName(playerId, rosters):

    #Get player name from current game roster

    for player in rosters:
        if player['playerId'] == playerId:
            first_name = player['firstName']['default']
            last_name = player['lastName']['default']

            return f"{last_name}, {first_name}", player['positionCode']
    return

def getShotCoords(shot, homeTeam):

    #Get coordinates and shot side from shot data

    x_coord = shot['xCoord']
    y_coord = shot['yCoord']
    
    if shot['eventOwnerTeamId'] == homeTeam:
        if shot['homeTeamDefendingSide'] == 'right':
            net_side = 'left'
        else:
            net_side = 'right'
    else:
        net_side = shot['homeTeamDefendingSide']

    return x_coord, y_coord, net_side

def calcAzimuthErrorMargin(x: int, y: int, net_side):

    #calculate margin of error in the azimuth dimension from puck to between posts

    if net_side == 'left':
        left_post = {'x': -89, 'y': -3}
        right_post = {'x': -89, 'y': 3}
    else:
        left_post = {'x': 89, 'y': 3}
        right_post = {'x': 89, 'y': -3}

    if x == left_post['x']:
        if y <= 3 and y >= -3:
            return math.pi / 2, (math.pi / 2) * -1
        else:
            return 0.0, 0.0

    angle_to_left = math.atan((left_post['y'] - y)/(left_post['x'] - x))
    angle_to_right = math.atan((right_post['y'] - y)/(right_post['x'] - x))

    return angle_to_left, angle_to_right

def calcElevationAngles(x: int, y: int, shot_side):

    #calculate the elevation margin of error from the puck to the corners of the crossbar
    #NOT CURRENTLY USED

    if shot_side == 'left':
        left_post = {'x': -89, 'y': -3}
        right_post = {'x': -89, 'y': 3}
    else:
        left_post = {'x': 89, 'y': 3}
        right_post = {'x': 89, 'y': -3}
    
    if x == left_post['x']:
        if y <= 3 and y >= -3:
            return math.pi / 2, math.pi / 2
        
    distance_to_left_post = math.sqrt(((left_post['x'] - x) ** 2) + ((left_post['y'] - y) ** 2))
    distance_to_right_post = math.sqrt(((right_post['x'] - x) ** 2) + ((right_post['y'] - y) ** 2))

    angle_to_left_post = math.atan(4 / distance_to_left_post)

    angle_to_right_post = math.atan(4 / distance_to_right_post)

    return angle_to_left_post, angle_to_right_post

def calcElevationToPoint(x: int, y: int, point_x: int, point_y: int):
    
    #calculate the elevation margin of error from the puck to a point on the crossbar.
    #NOT CURRENTLY USED
    
    if x == point_x:
        if y <= 3 and y >= -3:
            return math.pi / 2
        
    distance_to_point = math.sqrt(((point_x - x) ** 2) + ((point_y - y) ** 2))

    angle_to_point = math.atan(4 / distance_to_point)

    return angle_to_point

def calcSurfaceAreaErrorMargin(az_left, az_right, el_left, el_right):

    #calculate radial surface area margin of error from the puck to the net.
    #NOT CURRENTLY USED
        
    az_margin = az_left - az_right

    area_1 = (az_margin)*(math.cos(0) - math.cos(el_left))
    area_2 = (az_margin)*(math.cos(0) - math.cos(el_right))

    area = (abs(area_1) + abs(area_2)) / 2

    return abs(area)

def calcSurfaceAreaWithinPosts(az_left, az_middle, az_right, el_left, el_middle, el_right):

    #calculate radial surface area margin of error from the puck to the net.
    #NOT CURRENTLY USED

    area1 = calcSurfaceAreaErrorMargin(az_left, az_middle, el_left, el_middle)
    area2 = calcSurfaceAreaErrorMargin(az_middle, az_right, el_middle, el_right)

    area = abs(area1) + abs(area2)

    return abs(area)

def calcShotDifficulty(x, y, side):

    #calculate radial surface area margin of error from the puck to the net.
    #NOT CURRENTLY USED

    if x < -89 or x > 89:
        return 1 
    
    if side == 'left':
        net_x = -89
    else:
        net_x = 89

    az_left, az_right = calcAzimuthErrorMargin(x, y, side)

    el_left, el_right = calcElevationAngles(x, y, side)

    #print(f"{az_left}, {az_right}, {el_left}, {el_right}")

    if y < -3 or y > 3:
        area = calcSurfaceAreaErrorMargin(az_left, az_right, el_left, el_right )
    else:
        el_middle = calcElevationToPoint(x, y, net_x, y)
        #print(f"el_middle: {el_middle}")
        area = calcSurfaceAreaWithinPosts(az_left, 0, az_right, el_left, el_middle, el_right)

    difficulty = (math.pi - area) / math.pi

    return(difficulty)


def replaceIdsWithNames(player_db):

    #find player name based on ID.

    new_db = {}

    for player_id in player_db.keys():
        player_info = requests.get(f"https://api-web.nhle.com/v1/player/{player_id}/landing")

        # Make sure the shot was not on an empty net
        if player_info.status_code != 404:
            player_info = player_info.json()
            
            
            first_name = player_info['firstName']['default']
            last_name = player_info['lastName']['default']

            full_name = f"{last_name}, {first_name}"
            new_db[full_name] = player_db[player_id]

    return new_db

def isPowerPlay(shot, homeTeam):
    
    #Checks if shot taken was on the power play.
    #Situation code abcd. a = away goalie, b = away skaters, c = home skaters, d = home goalie. ex. 1451 = home 5v4 power play
    if shot['eventOwnerTeamId'] == homeTeam:
        if shot['situationCode'] == '1451' or shot['situationCode'] == '1351' or shot['situationCode'] == '1341' or shot['situationCode'] == '1450' or shot['situationCode'] == '1460' or shot['situationCode'] == '1360' or shot['situationCode'] == '1340':
            return True
        return False
    else:
        if shot['situationCode'] == '1541' or shot['situationCode'] == '1531' or shot['situationCode'] == '1431' or shot['situationCode'] == '0541' or shot['situationCode'] == '0641' or shot['situationCode'] == '0631' or shot['situationCode'] == '0431':
            return True
        return False



In [ ]:
player_db = {}
goalie_db = {}

SEASON = 2023
SEASON_TYPE = 2 # 1 = preseason, 2 = regular season, 3 = playoffs
GAMES_TRACKING = 1312
START_GAME = 1

# Go through every game inside the specified range.
for game in range(START_GAME, START_GAME + GAMES_TRACKING):
    
    # Insert appropriate number of zeros to get the right play-by-play url
    game_code = game
    if game < 10:
        game_code = f"000{game}"
    elif game < 100:
        game_code = f"00{game}"
    elif game < 1000:
        game_code = f"0{game}"

    gameId = f"{SEASON}0{SEASON_TYPE}{game_code}"

    # Pull play-by-play data from NHL API
    play_by_play = requests.get(f"https://api-web.nhle.com/v1/gamecenter/{gameId}/play-by-play")
    play_by_play = play_by_play.json()

    # Get team information
    home_team = play_by_play['homeTeam']['id']
    rosters = play_by_play['rosterSpots']

    # Grab shot details
    shot_details = getShotInfo(play_by_play)

    # Iterate through every shot from the game
    shots = 0
    for index, shot in shot_details.iterrows():
        
        if math.isnan(shot['shootingPlayerId']) == False or math.isnan(shot['scoringPlayerId']) == False:
            x, y, net_side = getShotCoords(shot, home_team)
            isPP = isPowerPlay(shot, home_team)

            # Get player id/name
            if shot['typeDescKey'] == "goal":
                player_id = int(shot['scoringPlayerId'])
            else:
                player_id = int(shot['shootingPlayerId'])
            player_id, position = getPlayerName(player_id, rosters)

            # Create new entry if player is not in db
            if player_id not in player_db:
                player_db[player_id] = {'position': position, 
                                        'avg. shot difficulty': 0.0, 'avg. goal difficulty': 0.0, 'adjusted shooting%': 0.0, 'shot-goal difference': 0.0,
                                        'avg. pp shot difficulty': 0.0, 'avg. pp goal difficulty': 0.0, 'adjusted pp shooting%': 0.0, 'pp shot-goal difference': 0.0,
                                        'avg. ev shot difficulty': 0.0, 'avg. ev goal difficulty': 0.0, 'adjusted ev shooting%': 0.0, 'ev shot-goal difference': 0.0,
                                        'shots': 0, 'pp shots': 0, 'ev shots': 0, 'goals': 0, 'pp goals': 0, 'ev goals': 0, "shooting%": 0.0, 'pp shooting%': 0.0, 'ev shooting%': 0.0,
                                        'shot locations (non-goals)': [], 'pp shot locations (non-goals)': [], 'ev shot locations (non-goals)': [], 'goal locations': [], 'pp goal locations': [], 'ev goal locations': [],
                                        }
            
            # Make sure shot is in the offensive zone and not on an empty net
            if math.isnan(shot['goalieInNetId']) == False and ((x >= 25 and net_side == 'right') or (x <= -25 and net_side == 'left')):

                # Calculate shot difficulty
                if x > 89 or x < -89:
                    shot_difficulty = 1
                else:
                    az_left, az_right = calcAzimuthErrorMargin(x, y, net_side)
                    margin = abs(az_left-az_right)
                    shot_difficulty = (math.pi - margin) / math.pi
                
                # Calculate metrics depending on whether or not the shot was a goal
                if shot['typeDescKey'] == "goal":
                    player_db[player_id]['avg. goal difficulty'] = ((player_db[player_id]['avg. goal difficulty'] *
                                                                    player_db[player_id]['goals'] + shot_difficulty) /
                                                                    ( player_db[player_id]['goals'] + 1))
                    player_db[player_id]['goals'] += 1
                    player_db[player_id]['goal locations'].append([x, y, net_side])
                    if isPP:
                        player_db[player_id]['avg. pp goal difficulty'] = ((player_db[player_id]['avg. pp goal difficulty'] *
                                                                    player_db[player_id]['pp goals'] + shot_difficulty) /
                                                                    ( player_db[player_id]['pp goals'] + 1))
                        player_db[player_id]['pp goals'] += 1
                        player_db[player_id]['pp goal locations'].append([x, y, net_side])
                    else:
                        player_db[player_id]['avg. ev goal difficulty'] = ((player_db[player_id]['avg. ev goal difficulty'] *
                                                                    player_db[player_id]['ev goals'] + shot_difficulty) /
                                                                    ( player_db[player_id]['ev goals'] + 1))
                        player_db[player_id]['ev goals'] += 1
                        player_db[player_id]['ev goal locations'].append([x, y, net_side])
                else:
                    player_db[player_id]['shot locations (non-goals)'].append([x, y, net_side])
                    if isPP:
                        player_db[player_id]['pp shot locations (non-goals)'].append([x, y, net_side])
                    else:
                        player_db[player_id]['ev shot locations (non-goals)'].append([x, y, net_side])

                player_db[player_id]['avg. shot difficulty'] = ((player_db[player_id]['avg. shot difficulty'] *
                                                                    player_db[player_id]['shots'] + shot_difficulty) /
                                                                    ( player_db[player_id]['shots'] + 1))
                player_db[player_id]['shots'] += 1
                if isPP:
                    player_db[player_id]['avg. pp shot difficulty'] = ((player_db[player_id]['avg. pp shot difficulty'] *
                                                                    player_db[player_id]['pp shots'] + shot_difficulty) /
                                                                    ( player_db[player_id]['pp shots'] + 1))
                    player_db[player_id]['pp shots'] += 1
                else:
                    player_db[player_id]['avg. ev shot difficulty'] = ((player_db[player_id]['avg. ev shot difficulty'] *
                                                                    player_db[player_id]['ev shots'] + shot_difficulty) /
                                                                    ( player_db[player_id]['ev shots'] + 1))
                    player_db[player_id]['ev shots'] += 1
                

# Calcuate more metrics
for player_id in player_db.keys():
    if player_db[player_id]['shots'] == 0:
        player_db[player_id]['shooting%'] = 0.0
        player_db[player_id]['pp shooting%'] = 0.0
        player_db[player_id]['ev shooting%'] = 0.0
    else:
        player_db[player_id]['shooting%'] = float(player_db[player_id]['goals']) / float(player_db[player_id]['shots'])
        if player_db[player_id]['pp shots'] != 0: player_db[player_id]['pp shooting%'] = float(player_db[player_id]['pp goals']) / float(player_db[player_id]['pp shots'])
        if player_db[player_id]['ev shots'] != 0: player_db[player_id]['ev shooting%'] = float(player_db[player_id]['ev goals']) / float(player_db[player_id]['ev shots'])

    player_db[player_id]['adjusted shooting%'] = player_db[player_id]['avg. shot difficulty'] * player_db[player_id]['shooting%']
    player_db[player_id]['shot-goal difference'] = player_db[player_id]['avg. goal difficulty'] - player_db[player_id]['avg. shot difficulty']
    player_db[player_id]['adjusted pp shooting%'] = player_db[player_id]['avg. pp shot difficulty'] * player_db[player_id]['pp shooting%']
    player_db[player_id]['pp shot-goal difference'] = player_db[player_id]['avg. pp goal difficulty'] - player_db[player_id]['avg. pp shot difficulty']
    player_db[player_id]['adjusted ev shooting%'] = player_db[player_id]['avg. ev shot difficulty'] * player_db[player_id]['ev shooting%']
    player_db[player_id]['ev shot-goal difference'] = player_db[player_id]['avg. ev goal difficulty'] - player_db[player_id]['avg. ev shot difficulty']

# Generate dataframe
player_db = pd.DataFrame.from_dict(player_db, orient='index')
player_db

In [28]:
#Sort data
player_db = player_db.sort_values(by = ['avg. shot difficulty'], ascending=True)
player_db



,position,avg. shot difficulty,avg. goal difficulty,adjusted shooting%,shot-goal difference,avg. pp shot difficulty,avg. pp goal difficulty,adjusted pp shooting%,pp shot-goal difference,avg. ev shot difficulty,...,ev goals,shooting%,pp shooting%,ev shooting%,shot locations (non-goals),pp shot locations (non-goals),ev shot locations (non-goals),goal locations,pp goal locations,ev goal locations
"Stolarz, Anthony",G,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0,0.0,0.0,0.0,[],[],[],[],[],[]
"Lajoie, Maxime",D,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0,0.0,0.0,0.0,[],[],[],[],[],[]
"Allen, Jake",G,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0,0.0,0.0,0.0,[],[],[],[],[],[]
"Shesterkin, Igor",G,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0,0.0,0.0,0.0,[],[],[],[],[],[]
"Steeves, Alex",C,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0,0.0,0.0,0.0,[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Jenik, Jan",R,0.976700,0.0,0.0,-0.976700,0.0,0.0,0.0,0.0,0.976700,...,0,0.0,0.0,0.0,"[[39, 40, right]]",[],"[[39, 40, right]]",[],[],[]
"Rifai, Marshall",D,0.976700,0.0,0.0,-0.976700,0.0,0.0,0.0,0.0,0.976700,...,0,0.0,0.0,0.0,"[[-39, -40, left]]",[],"[[-39, -40, left]]",[],[],[]
"Hayden, John",C,0.976863,0.0,0.0,-0.976863,0.0,0.0,0.0,0.0,0.976863,...,0,0.0,0.0,0.0,"[[-64, -38, left]]",[],"[[-64, -38, left]]",[],[],[]
"Crozier, Maxwell",D,0.980661,0.0,0.0,-0.980661,0.0,0.0,0.0,0.0,0.980661,...,0,0.0,0.0,0.0,"[[32, -14, right], [-74, 26, left], [-30, 18, ...",[],"[[32, -14, right], [-74, 26, left], [-30, 18, ...",[],[],[]


In [29]:
#save data to pickle

player_db.to_pickle('2023-24_player_sniper_metrics.pickle')

